In [1]:
import pandas as pd
from datetime import datetime
import ast
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
movies_df = pd.read_csv("movies.csv")
tags_df = pd.read_csv("tags.csv")
links_df = pd.read_csv("links.csv")
ratings_df = pd.read_csv("ratings.csv")

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
Names = movies_df.title.unique()

In [5]:
tags_df = tags_df.drop(columns = ["userId","timestamp"])
tags_df.head()

,movieId,tag
0,60756,funny
1,60756,Highly quotable
2,60756,will ferrell
3,89774,Boxing story
4,89774,MMA


def tags(movieId,tag):
    for i in movieId:
        for j in movieId:
            if i == j :
                tag+=tag
    return tag

In [6]:
#tags(tags_df.movieId, tags_df.tag)

In [7]:
#tags_df["tag]= tags_df[tags_df.tag.appl

In [8]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [9]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
movie_df = movies_df.merge(ratings_df,how = 'inner',on = 'movieId')
movie_df = movie_df.merge(tags_df,how = 'inner', on = 'movieId')
movie_df = movie_df.merge(links_df,how = 'inner',on = 'movieId')

In [11]:
movie_df.head()

,movieId,title,genres,userId,rating,timestamp,tag,imdbId,tmdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,pixar,114709,862.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,pixar,114709,862.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,fun,114709,862.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,pixar,114709,862.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,pixar,114709,862.0


In [12]:
from numpy.ma.core import shape
df = movie_df.drop(columns = ['timestamp','imdbId','tmdbId'])
print(shape(movie_df))
print(shape(df))

(9, 233213)
(6, 233213)


In [13]:
df = df.drop_duplicates()

In [14]:
df['tag'] = df[['movieId','title','genres','userId','rating','tag']].groupby(['movieId','title','genres','userId','rating'])['tag'].transform(lambda x: ','.join(x))
df = df[['movieId','title','genres','userId','rating','tag']].drop_duplicates()
df = df.groupby(['movieId','title','genres','userId','rating'])['tag'].apply(lambda x: ','.join(x)).reset_index()

In [15]:
df

,movieId,title,genres,userId,rating,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,"pixar,fun"
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,"pixar,fun"
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,"pixar,fun"
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,"pixar,fun"
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,"pixar,fun"
...,...,...,...,...,...,...
48282,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,380,4.0,"Emilia Clarke,star wars"
48283,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,414,3.5,"Emilia Clarke,star wars"
48284,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,514,3.0,"Emilia Clarke,star wars"
48285,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,586,5.0,"Emilia Clarke,star wars"


In [16]:
df_2 = df.pivot_table(columns = 'title', values = 'rating')
df_3 = df_2.transpose()
df_4= df.merge(right = df_3, how = "inner", on = 'title')
df_5 = df_4.drop(columns = ['rating_x'])
df_6 = df_5.drop_duplicates()
df_7 = df_6.sort_values(by= 'rating_y',  ascending = False )
df_7[df_7.genres.apply(lambda genres : genres == "Comedy")].head()

,movieId,title,genres,userId,tag,rating_y
26020,2295,"Impostors, The (1998)",Comedy,182,Screwball,4.500000
26021,2295,"Impostors, The (1998)",Comedy,474,Screwball,4.500000
31320,3341,Born Yesterday (1950),Comedy,367,remade,4.285714
31316,3341,Born Yesterday (1950),Comedy,156,remade,4.285714
31317,3341,Born Yesterday (1950),Comedy,188,remade,4.285714


In [17]:
df_7[df_7.genres.apply(lambda genres : genres == "Drama")].head()

,movieId,title,genres,userId,tag,rating_y
32959,3951,Two Family House (2000),Drama,414,In Netflix queue,5.000000
39560,7071,"Woman Under the Influence, A (1974)",Drama,318,In Netflix queue,5.000000
11285,670,"World of Apu, The (Apur Sansar) (1959)",Drama,474,India,4.750000
11284,670,"World of Apu, The (Apur Sansar) (1959)",Drama,410,India,4.750000
14384,1041,Secrets & Lies (1996),Drama,74,In Netflix queue,4.590909


In [18]:
df_7[df_7.genres.apply(lambda genres : genres == "Romance")].head()

,movieId,title,genres,userId,tag,rating_y
11090,638,Jack and Sarah (1995),Romance,474,babies,3.000000
33309,4024,"House of Mirth, The (2000)",Romance,603,Edith Wharton,2.166667
33308,4024,"House of Mirth, The (2000)",Romance,474,Edith Wharton,2.166667
33307,4024,"House of Mirth, The (2000)",Romance,108,Edith Wharton,2.166667


In [19]:
df_7[df_7.genres.apply(lambda genres : genres == "Adventure")].head()

,movieId,title,genres,userId,tag,rating_y
13765,941,"Mark of Zorro, The (1940)",Adventure,474,swashbuckler,3.75
13764,941,"Mark of Zorro, The (1940)",Adventure,448,swashbuckler,3.75


In [20]:
x = df.movieId.unique()
y = df_4.genres.unique()
z = df.userId.unique()
print(len(y))
print(len(x))
print(len(z))

370
1554
610


In [21]:
Matrix = df.pivot_table(values = 'rating',index ='userId', columns ='title')

In [22]:
Matrix_2 = Matrix.fillna(0)

In [23]:
Matrix.head()

title,(500) Days of Summer (2009),...And Justice for All (1979),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),"11'09""01 - September 11 (2002)",12 Angry Men (1957),127 Hours (2010),13 Going on 30 (2004),...,Young Frankenstein (1974),Z (1969),Zack and Miri Make a Porno (2008),Zelary (2003),Zelig (1983),Zero Dark Thirty (2012),Zombieland (2009),Zoolander (2001),Zulu (1964),eXistenZ (1999)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
similarity = cosine_similarity(Matrix_2,Matrix_2)
similarity_df = pd.DataFrame(similarity, index=Matrix.index, columns=Matrix.index)
similarity_df.head(10)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.022248,0.027001,0.247329,0.175428,0.179687,0.239219,0.183219,0.109528,0.037568,...,0.132618,0.222031,0.302658,0.097070,0.219546,0.256994,0.391477,0.402357,0.159757,0.251391
2,0.022248,1.000000,0.000000,0.005550,0.019876,0.018778,0.014947,0.034428,0.000000,0.137046,...,0.272150,0.023925,0.021624,0.000000,0.000000,0.049909,0.020071,0.054363,0.043528,0.177541
3,0.027001,0.000000,1.000000,0.029934,0.053606,0.060774,0.000000,0.055711,0.000000,0.000000,...,0.033617,0.061943,0.081843,0.000000,0.000000,0.078920,0.071453,0.084290,0.000000,0.023049
4,0.247329,0.005550,0.029934,1.000000,0.157897,0.121160,0.152475,0.089053,0.020935,0.054044,...,0.129211,0.161317,0.394010,0.074651,0.105807,0.293271,0.191333,0.209243,0.056928,0.199449
5,0.175428,0.019876,0.053606,0.157897,1.000000,0.422213,0.135942,0.439000,0.000000,0.055937,...,0.082175,0.485622,0.169959,0.321188,0.208863,0.154332,0.213102,0.191066,0.336987,0.097699
6,0.179687,0.018778,0.060774,0.121160,0.422213,1.000000,0.123031,0.499735,0.029753,0.053854,...,0.037388,0.590792,0.149866,0.475347,0.195673,0.168637,0.227978,0.238020,0.333892,0.103559
7,0.239219,0.014947,0.000000,0.152475,0.135942,0.123031,1.000000,0.152194,0.145474,0.193496,...,0.243500,0.169389,0.167311,0.092826,0.346620,0.296944,0.282795,0.431402,0.150449,0.315466
8,0.183219,0.034428,0.055711,0.089053,0.439000,0.499735,0.152194,1.000000,0.000000,0.046138,...,0.102257,0.644451,0.164143,0.407778,0.180021,0.150103,0.249696,0.258943,0.464998,0.134555
9,0.109528,0.000000,0.000000,0.020935,0.000000,0.029753,0.145474,0.000000,1.000000,0.102970,...,0.134798,0.016246,0.076319,0.040269,0.148669,0.105239,0.022715,0.125388,0.000000,0.104398


In [25]:
def features(row):
    return row['tag']+" "+row['genres']
df["features"] = df.apply(features, axis =1)

In [26]:
df.head()

,movieId,title,genres,userId,rating,tag,features
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,"pixar,fun","pixar,fun Adventure|Animation|Children|Comedy|..."
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,"pixar,fun","pixar,fun Adventure|Animation|Children|Comedy|..."
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,"pixar,fun","pixar,fun Adventure|Animation|Children|Comedy|..."
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,"pixar,fun","pixar,fun Adventure|Animation|Children|Comedy|..."
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,"pixar,fun","pixar,fun Adventure|Animation|Children|Comedy|..."
